# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846827040944                   -0.70    4.8         
  2   -7.852316497232       -2.26       -1.53    1.0   17.0ms
  3   -7.852615740511       -3.52       -2.56    1.5   18.5ms
  4   -7.852645984696       -4.52       -2.90    2.5   22.6ms
  5   -7.852646522839       -6.27       -3.22    1.2   64.7ms
  6   -7.852646677756       -6.81       -4.00    1.0   16.9ms
  7   -7.852646686330       -8.07       -5.30    1.8   19.8ms
  8   -7.852646686726       -9.40       -5.53    2.2   22.4ms
  9   -7.852646686730      -11.38       -6.42    1.0   17.2ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846810393123                   -0.70           4.5         
  2   -7.852540066243       -2.24       -1.64   0.80    2.2    214ms
  3   -7.852638209418       -4.01       -2.72   0.80    1.0   16.0ms
  4   -7.852646455246       -5.08       -3.33   0.80    2.2   20.9ms
  5   -7.852646682742       -6.64       -4.24   0.80    1.8   18.7ms
  6   -7.852646686534       -8.42       -4.91   0.80    2.0   19.2ms
  7   -7.852646686726       -9.72       -5.94   0.80    1.8   18.8ms
  8   -7.852646686730      -11.37       -7.10   0.80    2.2   20.9ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.385451e+01     3.763659e+00
 * time: 0.05268502235412598
     1     1.390855e+00     1.919916e+00
 * time: 0.22977685928344727
     2    -1.077664e+00     2.276322e+00
 * time: 0.2984459400177002
     3    -3.650115e+00     1.688990e+00
 * time: 0.3246159553527832
     4    -4.934106e+00     1.564010e+00
 * time: 0.3502950668334961
     5    -6.728707e+00     8.606188e-01
 * time: 0.37589287757873535
     6    -6.889741e+00     1.047871e+00
 * time: 0.393481969833374
     7    -7.503203e+00     4.196000e-01
 * time: 0.41112589836120605
     8    -7.643687e+00     2.265243e-01
 * time: 0.42868995666503906
     9    -7.715178e+00     1.603354e-01
 * time: 0.44629693031311035
    10    -7.788244e+00     1.281685e-01
 * time: 0.4640200138092041
    11    -7.819880e+00     7.959108e-02
 * time: 0.48162293434143066
    12    -7.843166e+00     3.268635e-02
 * time: 0.49927806854248047
    13    -7.849918e+00     3.670153e-02
 * time: 0.516

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846849132635                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645916162                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.47s
  3   -7.852646686730      -13.32       -7.25    108ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.55910130551548e-7
|ρ_newton - ρ_scfv| = 6.457720386247791e-8
|ρ_newton - ρ_dm|   = 6.3895769560949e-10
